In [1]:
import pandas as pd
import numpy as np

In [2]:
app_launch = pd.read_csv('../input/app_launch_log.txt',header=None,sep='\t',names=['user_id','launch_day'])
user_activity = pd.read_csv('../input/user_activity_log.txt',header=None,sep='\t',names=['user_id','activity_day','page','video_id','author_id','action_type'])
user_register = pd.read_csv('../input/user_register_log.txt',header=None,sep='\t',names=['user_id','register_day','register_type','device_type'])
video_create = pd.read_csv('../input/video_create_log.txt',header=None,sep='\t',names=['user_id','create_day'])
app_launch.sort_values(by=['user_id','launch_day'],inplace=True)

In [3]:
data1_user=user_register[(user_register['register_day'] >= 1) & (user_register['register_day'] <= 16)]
data2_user=user_register[(user_register['register_day'] >= 1) & (user_register['register_day'] <= 23)]

In [4]:
print(data1_user.shape)
print(len(np.unique(data1_user['user_id'])))
print(data2_user.shape)
print(len(np.unique(data2_user['user_id'])))

(23848, 4)
23848
(37446, 4)
37446


In [3]:
temp_create = video_create[(video_create['create_day'] >= 1) & (video_create['create_day'] <= 16)]
temp_launch = app_launch[(app_launch['launch_day'] >= 1) & (app_launch['launch_day'] <= 16)]
temp_activity = user_activity[(user_activity['activity_day'] >= 1) & (user_activity['activity_day'] <= 16)]
data2_temp_create = video_create[(video_create['create_day'] >= 8) & (video_create['create_day'] <= 23)]
data2_temp_launch = app_launch[(app_launch['launch_day'] >= 8) & (app_launch['launch_day'] <= 23)]
data2_temp_activity = user_activity[(user_activity['activity_day'] >= 8) & (user_activity['activity_day'] <= 23)]

In [6]:
print(len(np.unique(pd.concat([temp_create['user_id'],temp_launch['user_id'],temp_activity['user_id']]))))
print(len(np.unique(pd.concat([data2_temp_create['user_id'],data2_temp_launch['user_id'],data2_temp_activity['user_id']]))))

23848
33992


In [10]:
# 极差
def ptp(column):
    return max(column) - min(column)
# 方差除以评率
def var_divide_count(column):
    return np.var(column) / len(column)

def last_one(column):
    if len(column) < 1:
        return column
    return column.iloc[-1]
def last_one_minus_reg():
    return

def q25(column):
    return column.quantile(.25)


def q75(column):
    return column.quantile(.75)


def kurt(column):
    return column.kurt()

In [12]:
# APP 启动次数，平均次数，方差，启动天数，最大值，最小值，连续几天启动总次数，平均次数，某一天启动次数
# agg要使用的函数
func = ['min', 'max', 'mean', 'median', 'skew', 'std', 'mad', 'count', ptp, np.var, var_divide_count, last_one, q25, q75, kurt]
app = temp_launch.groupby('user_id')['launch_day'].agg(func).reset_index()
app = app.rename(columns={'min': 'app_min', 'max': 'app_max', 'mean': 'app_mean', 'median': 'app_median',
             'skew': 'app_skew', 'std': 'app_std', 'mad': 'app_mad', 'count': 'app_count',
             'ptp': 'app_ptp', 'var': 'app_var', 'var_divide_count': 'app_var_divide_count',
             'last_one': 'app_last_one', 'q25': 'app_q25', 'q75': 'app_q75', 'kurt': 'app_kurt'})

In [13]:
app.head()

,user_id,app_min,app_max,app_mean,app_median,app_skew,app_std,app_mad,app_count,app_ptp,app_var,app_var_divide_count,app_last_one,app_q25,app_q75,app_kurt
0,16,13,15,14.00,14.0,0.000000,1.000000,0.666667,3,2,1.000000,0.222222,15,13.50,14.50,NaN
1,98,16,16,16.00,16.0,NaN,NaN,0.000000,1,0,NaN,0.000000,16,16.00,16.00,NaN
2,105,12,16,14.25,14.5,-0.752837,1.707825,1.250000,4,4,2.916667,0.546875,16,13.50,15.25,0.342857
3,218,11,16,13.50,13.5,0.000000,1.870829,1.500000,6,5,3.500000,0.486111,11,12.25,14.75,-1.200000
4,225,7,7,7.00,7.0,NaN,NaN,0.000000,1,0,NaN,0.000000,7,7.00,7.00,NaN


In [26]:
launch_day_diff = pd.concat([app_launch['user_id'], app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)], axis=1)

In [44]:
launch_total_count = app_launch[['user_id']].groupby(['user_id']).size().rename('launch_total_count').reset_index()
launch_rest_day = (end_day - app_launch.groupby(['user_id'])['launch_day'].max() + 1).rename('launch_rest_day').reset_index()

In [50]:
    launch_rest_day['launch_average']=launch_total_count['launch_total_count'].div(launch_rest_day['launch_rest_day'])

In [46]:
launch_average=launch_total_count['launch_total_count']/launch_rest_day['launch_rest_day']

In [47]:
launch_average.head()

0     1.125000
1     0.142857
2     0.066667
3    15.000000
4     4.000000
dtype: float64

In [48]:
launch_total_count.head()

,user_id,launch_total_count
0,16,9
1,30,1
2,98,1
3,105,15
4,176,4


In [51]:
launch_rest_day.head()

,user_id,launch_rest_day,launch_average
0,16,8,1.125000
1,30,7,0.142857
2,98,15,0.066667
3,105,1,15.000000
4,176,1,4.000000


In [29]:
app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)

,launch_day_diff
0,NaN
1,NaN
2,5.0
3,2.0
4,1.0
5,6.0
6,6.0
7,2.0
8,2.0
9,NaN


In [27]:
launch_day_diff.head()

,user_id,launch_day_diff
0,383135,NaN
1,330986,NaN
2,330986,5.0
3,330986,2.0
4,330986,1.0


In [6]:
userPre = user_activity[user_activity['activity_day']>=24]
sub = userPre[['user_id']].drop_duplicates()
sub.to_csv("../submission/baseline0611.csv",encoding='utf-8',index=None,header=None)

In [31]:

df1 = pd.DataFrame({'A':[1,1,2,2],'B':[2,2,6,8],'C':[8,7,6,5]})
df2 = pd.DataFrame({'A':[3,1,6,9,56],'B':[4,2,9,10,76],'D':[1,2,3,4,25]})
#df = df1.merge(df2,how='outer',on=['A','B'],left_index=True)
#t = df1.groupby('A').agg({'C':'min'})

#t = df1.groupby(['A'])['C'].min().rename('T').reset_index()


In [36]:
t = df1.groupby('A').agg({'C':'min'})

In [37]:
t

,C
A,
1,7
2,5


In [38]:
df1

,A,B,C
0,1,2,8
1,1,2,7
2,2,6,6
3,2,8,5


In [41]:
t = df1.groupby('A').agg({'A':'mean'})

In [15]:
end_day = np.max(temp_launch['launch_day'])
start_day = np.min(temp_launch['launch_day'])

In [6]:
temp_launch.head()

,user_id,launch_day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


In [137]:
test=pd.DataFrame()
test['launch_day']=[1,2,3,4,5,12,13,15]

In [138]:
test

,launch_day
0,1
1,2
2,3
3,4
4,5
5,12
6,13
7,15


In [142]:
end_day=15
start_day=1

In [145]:
sorted(test['launch_day'].tolist())

[1, 2, 3, 4, 5, 12, 13, 15]

In [147]:
seq[0]

16

In [143]:
seq = sorted(line['launch_day'].tolist())
length = end_day - seq[0] + 1
seq = [x - start_day + 1 for x in seq]

In [150]:
launch_day_diff = pd.concat([temp_launch['user_id'], temp_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)], axis=1)

In [151]:
launch_day_diff.head()

,user_id,launch_day_diff
44140,16,NaN
44141,16,1.0
212269,16,1.0
223219,98,NaN
125555,105,NaN


In [152]:
launch_day_diff.groupby(['user_id'])['launch_day_diff'].max().rename('launch_day_diff_max').reset_index()

,user_id,launch_day_diff_max
0,16,1.0
1,98,NaN
2,105,2.0
3,218,1.0
4,225,NaN
5,336,5.0
6,422,NaN
7,607,NaN
8,711,NaN
9,746,NaN


In [155]:
%%time
app_launch.groupby('user_id').agg([min,max])

Wall time: 48.9 ms


launch_day    
               min max
user_id               
16              13  23
30              24  24
98              16  16
105             12  30
176             27  30
211             25  25
218             11  30
225              7  29
250             17  29
325             22  30
336              1  30
341             25  27
420             26  28
422              5   5
481             23  29
493             23  30
555             21  30
581             26  30
589             27  28
607              1   1
639             18  18
662             25  27
674             29  30
711              6  29
746             14  14
820             23  26
858             21  21
901             14  22
927             28  28
973              4  30
...            ...  ..
1366683         21  30
1366711          2   2
1366722         24  24
1366737         29  30
1366798         22  30
1366803         18  24
1366822         28  29
1366834          6  29
1366874         24  28
1366899          3  29
1366920         12  12
1366989         16  16
1367006         17  17
1367044          1  30
1367067         10  27
1367106         29  30
1367112         21  22
1367164         16  16
1367252         16  28
1367259         26  28
1367268         30  30
1367348         10  10
1367360         20  20
1367419         26  26
1367421          6  30
1367449         23  29
1367473          4   5
1367496         30  30
1367532         15  30
1367534         24  29

[51709 rows x 2 columns]

In [158]:
def test2(seq, user_id):
    df = pd.DataFrame()
    df['user_id'] = [user_id]
    df['max']=np.max(seq)
    df['min']=np.min(seq)
    return df

def test(line):
    user_id = line['user_id'].tolist()[0]
    test2(seq, user_id)

In [162]:
%%time
test = app_launch.groupby(['user_id'])['launch_day'].agg({'launch_day_duration': lambda x: (end_day - min(x) + 1)}).reset_index()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


Wall time: 668 ms


In [ ]:
%%time
test = app_launch.groupby(['user_id'])['launch_day'].agg({'launch_day_duration': lambda x: (end_day - min(x) + 1)}).reset_index()

In [1]:
from itertools import groupby
from operator import itemgetter

# 连续多少天启动
def get_continuous(data):
    ranges = []
    for k, g in groupby(enumerate(data), lambda x: x[0] - x[1]):
        group = (map(itemgetter(1), g))
        group = list(map(int, group))
        ranges.append(group[-1] - group[0] + 1)
    return max(ranges)

In [8]:
app_launch.groupby(['user_id'])['launch_day'].agg([get_continuous]).reset_index().rename()

,user_id,get_continuous
0,16,3
1,30,1
2,98,1
3,105,3
4,176,4
5,211,1
6,218,3
7,225,3
8,250,3
9,325,1


In [13]:
app_launch.groupby(['user_id']).size().rename('launch_total_count').reset_index()

,user_id,launch_total_count
0,16,9
1,30,1
2,98,1
3,105,15
4,176,4
5,211,1
6,218,19
7,225,14
8,250,12
9,325,4


In [14]:
app_launch.groupby(['user_id'])['launch_day'].size().rename('launch_total_count').reset_index()

,user_id,launch_total_count
0,16,9
1,30,1
2,98,1
3,105,15
4,176,4
5,211,1
6,218,19
7,225,14
8,250,12
9,325,4


In [ ]:
launch_day_diff = pd.concat([app_launch['user_id'], ], axis=1)

In [24]:
%%time
app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)

Wall time: 13.8 s


,launch_day_diff
44140,NaN
44141,1.0
212269,1.0
212270,3.0
212271,1.0
212272,1.0
127816,1.0
212273,1.0
212274,1.0
75755,NaN


In [23]:
%%time
pd.concat([app_launch['user_id'], app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)], axis=1)

Wall time: 14.2 s


,user_id,launch_day_diff
44140,16,NaN
44141,16,1.0
212269,16,1.0
212270,16,3.0
212271,16,1.0
212272,16,1.0
127816,16,1.0
212273,16,1.0
212274,16,1.0
75755,30,NaN


In [21]:
app_launch[app_launch['user_id']==330986]

,user_id,launch_day
83666,330986,1
83667,330986,2
167756,330986,3
1,330986,4
167757,330986,5
167758,330986,6
167759,330986,7
83668,330986,8
2,330986,9
167760,330986,10


In [25]:
app_launch[app_launch['user_id']==330986].diff()

,user_id,launch_day
83666,NaN,NaN
83667,0.0,1.0
167756,0.0,1.0
1,0.0,1.0
167757,0.0,1.0
167758,0.0,1.0
167759,0.0,1.0
83668,0.0,1.0
2,0.0,1.0
167760,0.0,1.0


In [26]:
video_create.sort_values(by=['user_id','create_day'])

,user_id,create_day
7951,98,16
9815,555,22
9814,555,24
21855,973,13
21849,973,14
21851,973,14
21852,973,14
21844,973,20
21843,973,25
21845,973,27


In [28]:
video_create.groupby(['user_id'])['create_day'].agg({'launch_day_duration': lambda x: len(np.unique(x))})

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,launch_day_duration
user_id,
98,1
555,2
973,6
1014,6
1180,1
1296,5
1400,1
1938,1
2040,3


In [30]:
video_create.groupby(['user_id'])['create_day'].agg('count')

user_id
98          1
555         2
973        15
1014        6
1180        1
1296        9
1400        4
1938        1
2040        8
2068        1
2276        5
2306        1
2439       17
2914       14
2941       10
3473        1
3650        3
3883        1
3897        6
3952        4
4019        3
4370        2
4527       51
4790        5
4800        2
4831        1
5527        2
5789        5
5861       17
5989        4
           ..
1362387     1
1362746    16
1362781     1
1362826    11
1362866     1
1363730     5
1363930     1
1364055     1
1364245     1
1364263     3
1364358     6
1364467     1
1364756     1
1364850     4
1364947     1
1365465     2
1365674     1
1366109     1
1366113     3
1366200     1
1366299     3
1366305     2
1366339     1
1366419     5
1366491     1
1366798     6
1366834     7
1367067     1
1367112     7
1367473    12
Name: create_day, Length: 7606, dtype: int64

In [4]:
user_activity.sort_values(by=['user_id','activity_day'],inplace=True)

,user_id,activity_day,page,video_id,author_id,action_type
496070,16,13,3,2911602,245430,0
522440,16,13,3,1066348,245430,0
522441,16,13,3,1066348,245430,0
815812,16,13,3,1277921,280089,0
1289214,16,13,3,2565669,280089,0


In [5]:
user_activity.head(15)

,user_id,activity_day,page,video_id,author_id,action_type
496070,16,13,3,2911602,245430,0
522440,16,13,3,1066348,245430,0
522441,16,13,3,1066348,245430,0
815812,16,13,3,1277921,280089,0
1289214,16,13,3,2565669,280089,0
2131571,16,13,3,2653595,280089,0
4277700,16,13,3,3355338,280089,0
4550265,16,13,3,3860100,280089,0
6024364,16,13,3,1747202,280089,0
7160265,16,13,3,1957749,103001,0
